In [1]:
import sys

sys.path.insert(0,'../')
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.utils import shuffle
import pandas as pd
from itertools import chain
import numpy as np
from scipy.sparse import csr_matrix
from tqdm import tqdm
from multiprocessing import Pool
from scipy.sparse import vstack
from functools import reduce

pd.set_option("display.max_columns", 150)

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((  loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])

In [ ]:
all_data = pd.concat([pd.read_csv('../data/features.csv.zip',compression='zip'),\
                     pd.read_csv('../data/features_test.csv.zip',compression='zip')])

In [ ]:
all_data.first_blood_team = all_data.first_blood_team.replace(0.0,-1.0)
all_data.first_blood_team = all_data.first_blood_team.fillna(0.0)

all_data.first_blood_player2 = all_data.first_blood_player2.fillna(0.0)
all_data.first_blood_player1 = all_data.first_blood_player1.fillna(0.0)
all_data.first_blood_player1 = all_data.first_blood_player1.astype(int)
all_data.first_blood_player2 = all_data.first_blood_player2.astype(int)

In [ ]:
all_data.lobby_type = all_data.lobby_type.replace(7,'ranked')
all_data.lobby_type = all_data.lobby_type.replace(0,'public_matchmaking')
all_data.lobby_type = all_data.lobby_type.replace(1,'practice')

onehot_cols = ['lobby_type',]

In [ ]:
all_data = pd.get_dummies(all_data, prefix=None, prefix_sep='_', dummy_na=False, columns=onehot_cols, sparse=False, drop_first=False)

In [ ]:
for feat in list(all_data.columns):
    if feat.endswith('level'):
        prefix = feat[:3]
        all_data[prefix+'levhero'] = all_data[feat].astype(str).str.cat(all_data[prefix+'hero'].astype(str),sep='_'+prefix[0]+'_')

In [ ]:
all_hero_unique_vals = list(all_data.r1_hero.unique())

def get_hero_ordered_level(level,side,max_level=7):
    return [(side if j<=int(level) else 0.0) for j in range(max_level+1)]

def encode_hero(herolev_val,heroes_total=108,max_level=7):
    hero_level,hero_side,hero_index = herolev_val.split('_')
    
    side_multiplier = 1.0 if hero_side=='r' else -1.0
    
    res = [get_hero_ordered_level(hero_level,side_multiplier) if int(hero_index)==i else [0.0 for i in range(max_level+1)]
            for i in all_hero_unique_vals]
    return csr_matrix([r for r in chain(*res)])
    
def encode_df_col(df_col):
    return all_data[df_col].apply(encode_hero)

In [ ]:
# p1 = Pool(4)

# levhero_cols = [col for col in all_data.columns if col.endswith('levhero')]

# all_heroes_vectors_accumulated = p1.map(encode_df_col, levhero_cols)

# v_stacked = [vstack(rows) for rows in all_heroes_vectors_accumulated]

# v_stacked_additioned  = reduce(lambda x,y: x + y, v_stacked)

# save_sparse_csr('./sparse_herolev.csr',v_stacked_additioned)

In [ ]:
v_stacked_additioned = load_sparse_csr('./sparse_herolev.csr.npz')